In [42]:
import json
import datetime
import requests
import pandas as pd

In [125]:
def get_commanders_for_players(gre_to_client_event):
    def _get_scryfall_name_from_arena_id(id: int):
        resp = json.loads(requests.get(f"https://api.scryfall.com/cards/arena/{id}").text)
        return resp.get("name", id)
        
    commanders = [
        message for message in gre_to_client_event["greToClientEvent"]["greToClientMessages"]
        if message["type"] == "GREMessageType_GameStateMessage"
    ][0]["gameStateMessage"]["gameObjects"]
    
    return [
        {
            "match_id": [
                message for message in gre_to_client_event["greToClientEvent"]["greToClientMessages"]
                if message["type"] == "GREMessageType_GameStateMessage"
            ][0]["gameStateMessage"]["gameInfo"]["matchID"],
            "player_seat_id": player["ownerSeatId"],
            "commander": _get_scryfall_name_from_arena_id(player["grpId"])
        } for player in commanders]

In [120]:
BASE_PATH = "/mnt/c/Users/chris"

In [247]:
deck_data = []
malformed_deck_data = []
match_data = []
malformed_data = []

commander_data = []
malformed_commander_data = []

In [248]:
_state_change_reached = False
_counter = 0

with open(f"{BASE_PATH}/AppData/LocalLow/Wizards Of The Coast/MTGA/Player.log", "r") as f:
    for line in f.readlines():
        if _state_change_reached is True:
            _counter += 1
        if line == '[UnityCrossThreadLogger]STATE CHANGED {"old":6,"new":7}\n':
            _state_change_reached = True
            _counter = 0
            
        if _state_change_reached and _counter == 2:
            try:
                commander_data.append(json.loads(line))
            except:
                malformed_commander_data.append(line)
        elif line.startswith('{"Summary":{"DeckId":') is True:
            try:
                deck_data.append(json.loads(line))
            except:
                malformed_deck_data.append(line)
        elif '"stateType": "MatchGameRoomStateType_MatchCompleted"' in line:
            try:
                match_data.append(json.loads(line))
            except:
                malformed_data.append(line)

In [249]:
assert len(deck_data) == len(match_data)

In [252]:
match_data_parsed = []
for deck, match in zip(deck_data, match_data):
    player_seat = [
        player["teamId"] for player in match["matchGameRoomStateChangedEvent"]["gameRoomInfo"]["gameRoomConfig"]["reservedPlayers"]
        if player["playerName"] == "spantz"
    ][0]
    
    _match_data = {
        "game_id": match["matchGameRoomStateChangedEvent"]["gameRoomInfo"]["gameRoomConfig"]["matchId"],
        "timestamp": datetime.datetime.fromtimestamp(int(match["timestamp"]) / 1000),
        "deck_name": deck["Summary"]["Name"],
        "opponent": [
            player["playerName"] for player in match["matchGameRoomStateChangedEvent"]["gameRoomInfo"]["gameRoomConfig"]["reservedPlayers"]
            if player["playerName"] != "spantz"
        ][0],
        "game_won": [
            result["winningTeamId"] for result in match["matchGameRoomStateChangedEvent"]["gameRoomInfo"]["finalMatchResult"]["resultList"]
            if result["scope"] == "MatchScope_Game"
        ][0] == player_seat,
        "game_result_reason": [
            result["reason"] for result in match["matchGameRoomStateChangedEvent"]["gameRoomInfo"]["finalMatchResult"]["resultList"]
            if result["scope"] == "MatchScope_Game"
        ][0],
        "player_seat": player_seat,
        "opponent_seat": 3 - player_seat,
    }

    match_data_parsed.append(_match_data)

In [253]:
commander_data_parsed = []
for event in commander_data:
    commander_data_parsed.extend(get_commanders_for_players(event))

commander_df = pd.DataFrame(commander_data_parsed)

In [236]:
match_df = pd.DataFrame(match_data_parsed)

In [237]:
if not commander_df.empty:
    match_df = match_df.merge(
        commander_df,
        left_on=["game_id", "player_seat"],
        right_on=["match_id", "player_seat_id"],
        how="left"
    )
    
    match_df = match_df.merge(
        commander_df,
        left_on=["game_id", "opponent_seat"],
        right_on=["match_id", "player_seat_id"],
        suffixes=["", "_opponent"],
        how="left"
    )

    match_df = match_df.drop([
        "player_seat_id", "match_id",
        "match_id_opponent", "player_seat_id_opponent"],
        axis=1
    )
else:
    match_df["commander"] = None
    match_df["commander_opponent"] = None
match_df = match_df.drop(["player_seat", "opponent_seat"], axis=1)

In [238]:
match_df

,game_id,timestamp,deck_name,opponent,game_won,game_result_reason,commander,commander_opponent
0,060a7c53-1ded-48e3-b56b-6ba29d8b9c30,2025-04-11 21:06:28.203,You've Got Gnomes,Doc5,True,ResultReason_Concede,"Anim Pakal, Thousandth Moon","Roxanne, Starfall Savant"
1,50797949-84b0-4a4c-b515-040fcbe063ce,2025-04-11 21:19:40.122,You've Got Gnomes,KingGirth,False,ResultReason_Concede,"Anim Pakal, Thousandth Moon","Atraxa, Praetors' Voice"
2,8c3c53eb-5e12-495b-9141-1a28a646712e,2025-04-11 21:23:20.411,You've Got Gnomes,あかさTaNaHa,True,ResultReason_Game,"Anim Pakal, Thousandth Moon","Atraxa, Grand Unifier"
3,074f5b0c-9958-46d1-9f3c-1941a767d85c,2025-04-11 21:28:10.203,You've Got Gnomes,Shade.BDE,True,ResultReason_Game,"Anim Pakal, Thousandth Moon","Imoti, Celebrant of Bounty"
4,4e85e775-baf3-40bb-8d31-8fcf9781c8a1,2025-04-11 21:29:47.161,You've Got Gnomes,dhammerliveson,True,ResultReason_Concede,NaN,NaN
5,79df1b57-cd53-4981-a80d-d0c419c86078,2025-04-11 21:37:14.050,You've Got Gnomes,RUCKUS,False,ResultReason_Game,"Anim Pakal, Thousandth Moon","Kinnan, Bonder Prodigy"
6,2ebcd41f-8dbe-4707-80c2-46c9ef761719,2025-04-11 21:41:46.684,You've Got Gnomes,Lugaid,False,ResultReason_Concede,"Anim Pakal, Thousandth Moon",95516
7,48f1372e-cd0c-4aff-b30a-f950cfa44eba,2025-04-11 21:44:43.132,You've Got Gnomes,Allred1000,False,ResultReason_Concede,NaN,NaN
8,2db616a2-e0c7-4708-9d5b-2b66389955b3,2025-04-11 21:49:43.784,You've Got Gnomes,lord jore,True,ResultReason_Game,"Anim Pakal, Thousandth Moon","Narset, Enlightened Master"
9,dd0dcc83-4e79-4eec-b56e-ab7533a060cf,2025-04-11 21:59:03.442,You've Got Gnomes,Psillias2,False,ResultReason_Game,NaN,NaN


In [239]:
sum(match_df["game_won"])/match_df.shape[0]

0.6190476190476191

In [240]:
match_df.to_csv(f"{BASE_PATH}/Desktop/mtga_match_log.csv", index=False, header=False, mode="a")
match_df.to_csv(f"mtga_match_log.csv", index=False, header=False, mode="a")

In [255]:
import shutil
import os
import time


source_path = f"{BASE_PATH}/AppData/LocalLow/Wizards Of The Coast/MTGA/Player.log"
destination_path = f"{BASE_PATH}/AppData/LocalLow/Wizards Of The Coast/MTGA/Player_{time.time_ns()}.log"

# Ensure the source file exists
if os.path.exists(source_path):
    shutil.move(source_path, destination_path)
    print(f"File moved from {source_path} to {destination_path}")
else:
    print(f"Source file {source_path} not found.")

File moved from /mnt/c/Users/chris/AppData/LocalLow/Wizards Of The Coast/MTGA/Player.log to /mnt/c/Users/chris/AppData/LocalLow/Wizards Of The Coast/MTGA/Player_1744431787553069525.log
